In [ ]:
# !pip install vaderSentiment


In [2]:
from modules import components
from unidecode import unidecode
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd

Since enemies and friends dont seem to occur in the same snetence(or 20 word contexts) it willbe safe to calculate sentiments per sentence and then propogate the same to the characters in it
Besides, we are only calculating the sentiments for characters relations, in each book. Therefore, the sentiment scores get averaged, so that the overall relationship makes sense

In [3]:
ch = components.get_char_list()[0]
characters = [unidecode(c.lower()) for c in ch]

### VADER/BERT/Distilbert

For each book:
For each chapter, for each sentence:
Get sentiment score
get char tuples for the sentence, add sentiment score for that tuple,
average by occurences, for both book wise and chap wise data

## VADER

In [4]:
df = pd.read_csv("../data/csv/c_b_t.csv", encoding='utf-8')
df.head()

,Chapter,Book,Text
0,Invocation,Book,"Praise to Válmíki,(2)bird of charming song,(3)..."
1,Canto I. Nárad.,Book I.,"OM.(8) To sainted Nárad, prince of those Wh..."
2,Canto II. Brahmá's Visit,Book I.,"Válmíki, graceful speaker, heard, To highest..."
3,Canto III. The Argument.,Book I.,The hermit thus with watchful heed Received ...
4,Canto IV. The Rhapsodists.,Book I.,"When to the end the tale was brought, Rose i..."


In [5]:
def sentiment_scores(sentence):
# Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
# polarity_scores method of SentimentIntensityAnalyzer
# oject gives a sentiment dictionary.
# which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
    # print("Overall sentiment dictionary is : ", sentiment_dict)
    # print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative")
    # print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral")
    # print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive")
    # print("Sentence Overall Rated As", end = " ")
    # decide sentiment as positive, negative and neutral
    # if sentiment_dict['compound'] >= 0.05 :
    #     print("Positive")
    # elif sentiment_dict['compound'] <= - 0.05 :
    #     print("Negative")
    # else :
    #     print("Neutral")
    return sentiment_dict


In [6]:
sentiment_scores("x loves y")


{'neg': 0.0, 'neu': 0.351, 'pos': 0.649, 'compound': 0.5719}

In [7]:
ch = components.get_char_list()[0]
characters = [unidecode(c.lower()) for c in ch]


In [8]:
sentiment_scores(df["Text"][0])

{'neg': 0.025, 'neu': 0.663, 'pos': 0.313, 'compound': 0.9944}

In [9]:
# returns relations dict with all relations in txt
def get_sentiments(raw, characters, window):
    raw = re.sub("'s", "", raw)
    raw = unidecode(raw).strip().lower() #list of words in canto
    txt = raw.split()

    relations = {}
    for word in txt:
            word = word.lower()
            if word in characters:
                source = word
                t_idx = txt.index(source)
                context = txt[t_idx : int(t_idx + window/2)]
                con_words = [cha for cha in context if cha in characters]
                score = sentiment_scores(" ".join(context))['compound']
                # print(score)
                for target in con_words:
                    if(source != target):
                        if ((source, target) in relations.keys()):      
                            relations[(source, target)] += score
                        elif ((target, source) in relations.keys() ):
                            relations[(target, source)] += score
                        else:
                            relations[(source, target)] = score
    return relations
# get_sentiments(df["Text"][1], characters, 50)
#maricha ravan negative relation incorrect
# ('vibhishan', 'sita'): -0.631,

# therefore, less char relationship sentiment, more circumstances being negative/+ve

In [58]:
ch1 = []
ch2 = []
wt = [] #sum of sentiments# needs to be averaged
boo = [] 

# relations must be 1 per book
# therefore all relations(weights) obtained within a local context and withina  broader context of a canto must be added

ch = components.get_char_list()[0]
characters = [unidecode(c.lower()) for c in ch]

bnc = components.books_to_chaps()
book_ends = [bnc[key][len(bnc[key]) - 1] for key in bnc.keys()] #the last chaptrer in each book

window = 50 #of words(0 indexed, iteration, maximum non inclusive)(context will be window/2 on either side)
#so, basically 1/2
#taking the words from this df will give canto context also, that the books list wont be able to give
# ie., this wont pick up on the last word in a chapter and the first word in the next
# could be skipped though- but good practice considering most epics have a habit of starting each chap with unrelated stories

# this also means that the last chapter of each book must be identified so that the canto wise relations can be added
b_relations = {}
book = 0
for idx, row in df.iterrows():
    raw = row["Text"]
    
    if unidecode(row["Chapter"]).lower().replace(".", "") != unidecode(book_ends[book]).lower().replace(".", ""): 
        #will need to add stuff from the last canto
         #need tuple of relations with weights: this is for each bok
        d_temp = get_sentiments(raw, characters, window) #chapter wise relations
        for key in d_temp:
            if key in b_relations:
                b_relations[key] += d_temp[key]
            else:
                b_relations[key] = d_temp[key]
        
        
    else:
        last_temp = get_sentiments(raw, characters, window)
        for key in last_temp:
            if key in b_relations:
                b_relations[key] += last_temp[key]
            else:
                b_relations[key] = last_temp[key]
        # print(b_relations)
        for key in b_relations: # all relations in a canto
            ch1.append(key[0])
            ch2.append(key[1])
            wt.append(b_relations[key])
            boo.append(book + 1)
        book += 1
        b_relations = {}
    
len(ch1) == len(ch2) == len(wt) == len(boo)


True

In [71]:
#**************run only once ************

co = pd.read_csv("../data/csv/cooccs_books.csv")
if co.shape[0] == len(wt):
    co.insert(co.shape[1], 'vader', wt)
co["vader"] = co["vader"]/co["Weight"]
co.to_csv("../data/csv/sentiments_books.csv", index=False)


In [73]:
co = pd.read_csv("../data/csv/sentiments_books.csv")
co.head()

,Source,Target,Weight,Book,vader
0,valmiki,rama,1,1,0.8172
1,kaikeyi,lakshman,1,2,0.8658
2,janak,rohini,1,2,0.9168
3,rohini,rama,1,2,0.3919
4,sringavera,ganga,1,2,0.5106


In [10]:
ch1 = []
ch2 = []
wt = []
canto = [] 
boo = []
c_no = []

ch = components.get_char_list()[0]
characters = [unidecode(c.lower()) for c in ch]

window = 40 #of words(0 indexed, iteration, maximum non inclusive)(context will be window/2 on either side)
#so, basically 1/2
for idx, row in df.iterrows():
    c_relations = {}
    raw = row["Text"]    
    d_temp = get_sentiments(raw, characters, window) #chapter wise relations
    for key in d_temp:
        if key in c_relations:
            c_relations[key] += d_temp[key]
        else:
            c_relations[key] = d_temp[key]
                
    for key in c_relations: # all relations in a canto
        ch1.append(key[0])
        ch2.append(key[1])
        wt.append(c_relations[key])
        canto.append(row["Chapter"])
        boo.append(row["Book"])
        c_no.append(idx)

len(ch1) == len(ch2) == len(wt) == len(canto) == len(boo) == len(c_no)
    


True

In [11]:

co = pd.read_csv("../data/csv/cooccs_chaps.csv")
if co.shape[0] == len(wt):
    co.insert(co.shape[1], 'vader', wt)
co["vader"] = co["vader"]/co["Weight"]
co.insert(co.shape[1], 'Canto_no', c_no)
co.to_csv("../data/csv/sentiments_chaps.csv", index=False)


In [12]:
co = pd.read_csv("../data/csv/sentiments_chaps.csv")
co.head()

,Source,Target,Weight,Canto,Book,vader,Canto_no
0,valmiki,rama,1,Invocation,Book,0.8172,0
1,kaikeyi,lakshman,1,Canto I. Nárad.,Book I.,0.8658,1
2,janak,rohini,1,Canto I. Nárad.,Book I.,0.9168,1
3,sringavera,ganga,1,Canto I. Nárad.,Book I.,0.5106,1
4,bharadvaja,chitrakuta,1,Canto I. Nárad.,Book I.,0.5106,1


In [13]:
co.shape

(2209, 7)

## For sentiment models:
https://huggingface.co/blog/sentiment-analysis-python <br>
https://huggingface.co/bhadresh-savani/distilbert-base-uncased-emotion?text=I+feel+a+bit+let+down